# Understanding `george`

For a comprehensive review of Gaussian Processes, see [this page](https://github.com/bmorris3/gp_interact) by Dr. Brett Morris.

In this notebook, I will try to fit KELT-11b transit observed by CHEOPS -- using GPs to decorrelate against various parameters, time and roll-angle. And in doing so, I will play with the functionalities of `george`.

We use `dynesty` here for fitting!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import george
from scipy.optimize import minimize as mz
import emcee
import corner
import batman
from george.modeling import Model
import dynesty
import scipy

Let's first see, how our data looks like,

In [ ]:
tim, fl, fle, roll = np.loadtxt('kelt-11.dat', usecols=(0,1,2,3), unpack=True)

fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1,
                                    figsize=(16, 9), constrained_layout = True)

ax0.set_title('Light curve', fontsize=16)
ax0.errorbar(tim, fl, yerr=fle, fmt='.', c='cornflowerblue')
ax0.set_xlim([np.min(tim), np.max(tim)])
ax0.set_xlabel('Time (in arbitrary units)')
ax0.set_ylabel('Normalised flux')
ax0.grid()
#ax0.legend()

ax1.set_title('Trend with Roll angle', fontsize=16)
ax1.errorbar(roll, fl, yerr=fle, fmt='.', c='orangered')
ax1.set_xlim([np.min(roll), np.max(roll)])
ax1.set_xlabel('Roll Angle')
ax1.set_ylabel('Flux')
ax1.grid()
#plt.legend()

#fig.suptitle('Theoretical and empirical distributions of the estimate of parameters', fontsize=16)

Okay, so one can now clearly see a trend with roll angle in flux -- we want to model this lines using Gaussian processes. In addition to this, there is a trend with time, that is why we will use a multidimensional kernel to detrend this data and also fit a transit model simultaneously. We will use transit model from `batman` in this case.

In [ ]:
# Transit model from george.modeling import Model

def transit_model_batman(rp, a, inc, q1, q2, t):
    params = batman.TransitParams()
    params.t0 = 0.50187268                       #time of inferior conjunction
    params.per = 4.73620495                      #orbital period
    params.rp = rp                               #planet radius (in units of stellar radii)
    params.a = a                                 #semi-major axis (in units of stellar radii)
    params.inc = inc                             #orbital inclination (in degrees)
    params.ecc = 0.                              #eccentricity
    params.w = 90.                               #longitude of periastron (in degrees)
    params.u = [q1, q2]                          #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"
    m = batman.TransitModel(params, t)           #initializes model
    flux = m.light_curve(params)                 #calculates light curve
    return flux

class transit(Model):
    parameter_names = ("rp", "a", "inc", "q1", "q2")   # This should be 'parameter_names' only

    # It's mandatory to give this function name "get_value"
    def get_value(self, tim):
        tim = tim.flatten()
        flx = transit_model_batman(self.rp, self.a, self.inc, self.q1, self.q2, tim)
        return flx

model = transit(rp=0.01, a=5., inc=80., q1=0.5, q2=0.5)

gp = george.GP(np.var(fl) * george.kernels.Matern32Kernel(1.5), mean=model)
gp.compute(tim, fle)

In [ ]:
print(gp.get_parameter_names())      # To print parameter names
print(gp.get_parameter_vector())     # To print their present values

In [ ]:
np.var(fl)

In [ ]:
def log_like(p):
    gp.set_parameter_vector(p)
    return gp.log_likelihood(fl, quiet=True)

def prior_transform(u):
    """Transforms the uniform random variables `u ~ Unif[0., 1.)`
    to the parameters of interest."""
    x = np.array(u)  # copy u
    # Rp
    x[0] = u[0]/10.
    # a/R
    x[1] = 9.*u[1] + 1.
    # inc
    x[2] = scipy.stats.norm.ppf(u[2], loc=85., scale=10.)
    # q1
    x[3] = 2.*u[3] - 1.
    # q2
    x[4] = 2.*u[4] - 1.
    # First kernel parameter
    x[5] = scipy.stats.loguniform.ppf(u[5], a=1.e-7, b=1.e-4)
    # Second kernel parameter
    x[6] = scipy.stats.loguniform.ppf(u[6], a=1.e-3, b=10.)
    return x

In [ ]:
sampler = dynesty.NestedSampler(loglikelihood=log_like, prior_transform=prior_transform, ndim=7, nlive=500, bound='multi', sample='rwalk')
sampler.run_nested()
results = sampler.results